# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [2318]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

from collections import Counter

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [2319]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        self.feature_importances_divisor_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.I_function = self.__gini_i
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.I_function = self.__entropy_i
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.I_function = self.__misclass_i
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    # G_function = [ I(SL)*|SL|+I(SR)*|SR| for all thres ]
    # node_impurity = I(S)*|S|

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s[0] - np.sum(l_c ** 2, axis=1) / l_s.flatten() \
            + r_s[0] - np.sum(r_c ** 2, axis=1) / r_s.flatten()

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.nansum(l_c * np.log(l_c / l_s), axis=1) \
            - np.nansum(r_c * np.log(r_c / r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return l_s[0] - np.max(l_c, axis=1) \
            + r_s[0] - np.max(r_c, axis=1)

    def __gini_i(self, y):
        counts = np.array(list(Counter(y).values()))
        return y.size - np.sum(counts ** 2) / y.size

    def __entropy_i(self, y):
        counts = np.array(list(Counter(y).values()))
        return -np.nansum(counts * np.log(counts / y.size))

    def __misclass_i(self, y):
        return y.size - Counter(y).most_common(1)[0][1]

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:max(int(np.sqrt(n_feature)), 1)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:max(int(np.log2(n_feature)), 1)]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # упорядочивает x (вектор значений фичи в выборке)
        # и в соответствии с ним y (вектор значений целевой переменной)
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # режет y с двух сторон и находит в массиве y_splitted_sorted
        # все границы между последовательностями одинаковых чисел
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)
        if len(r_border_ids) == 0:
            return np.inf, None

        # переводит выборку в формат one-hot-encoding
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # строит разбиение выборки на две части и размеры обоих частей
        # для каждого варианта выбора значения порога
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # считает меру неопреденности для разбиения по каждому значению порога
        # и находит разбиение, при котором эта неопределенность минимальна
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # возвращает меру неопределенности полученного порога и сам порог
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        if not(self.max_depth is not None and depth >= self.max_depth
                or Counter(y).most_common(1)[0][1] / y.size
               >= self.sufficient_share):
            n_feature = x.shape[1]
            feature_ids = self.get_feature_ids(n_feature)
            gs, thresholds = zip(*map(lambda f:
                                      self.__find_threshold(x[:, f], y),
                                      feature_ids))
            gs = np.array(gs)
            idx = np.argmin(gs)
            feature_id = feature_ids[idx]
            threshold = thresholds[idx]
            self.tree[node_id] = (self.__class__.NON_LEAF_TYPE,
                                  feature_id,
                                  threshold)
            x_l, x_r, y_l, y_r = self.__div_samples(x, y,
                                                    feature_id,
                                                    threshold)
            if y_l.size >= self.min_samples_split \
                    and y_r.size >= self.min_samples_split:
                imp = self.I_function(y) - gs[idx]
                self.feature_importances_[feature_id] += imp
                self.feature_importances_divisor_ += imp
                self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
                self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
                return
        pred_class = Counter(y).most_common(1)[0][0]
        pred_prob = y[y == pred_class].size / y.size
        self.tree[node_id] = (self.__class__.LEAF_TYPE,
                              pred_class,
                              pred_prob)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros((x.shape[1]))
        self.feature_importances_divisor_ = 0
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return y_pred[y_pred == y_test].shape[0] / y_pred.shape[0]

    def feature_importances(self):
        return self.feature_importances_ / self.feature_importances_divisor_

In [2320]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [2321]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [2322]:
%time clf.fit(X_train, y_train)

CPU times: user 2.91 ms, sys: 66 µs, total: 2.97 ms
Wall time: 2.93 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [2323]:
%time my_clf.fit(X_train, y_train)

CPU times: user 6.33 ms, sys: 7.93 ms, total: 14.3 ms
Wall time: 17.6 ms


## Проверка качества работы на wine

In [2324]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

In [2325]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

## Подготовка данных Speed Dating Data 

In [2326]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df_orig = pd.read_csv('data/speed-dating-experiment/Speed Dating Data.csv', 
                 encoding='cp1251')

In [2327]:
# %%pycodestyle

columns_select = [
    "gender",
    "round",
    "age_o",
    "race_o",
    "condtn",
    "int_corr",
    "samerace",
    "pf_o_sin",
    "pf_o_int",
    "pf_o_fun",
    "pf_o_amb",
    "pf_o_sha",
    "career_c",
    "attr",
    "sinc",
    "intel",
    "fun",
    "amb",
    "shar",
    "like",
    "prob",
    "met",
    "match_es",
    "attr_o",
    "sinc_o",
    "intel_o",
    "fun_o",
    "amb_o",
    "shar_o",
    "prob_o",
    "met_o",
    "goal",
    "match"]

df = df_orig.loc[range(df_orig.shape[0]), columns_select].dropna()
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)

In [2328]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_features='sqrt')
clf = DecisionTreeClassifier(min_samples_split=2, max_features='sqrt')

## Проверка скорости работы на Speed Dating Data

In [2329]:
%time clf.fit(X_train, y_train)

CPU times: user 15.5 ms, sys: 0 ns, total: 15.5 ms
Wall time: 16.3 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [2330]:
%time my_clf.fit(X_train, y_train)

CPU times: user 79.2 ms, sys: 60 µs, total: 79.3 ms
Wall time: 78.1 ms


## Проверка качества работы на Speed Dating Data 

In [2331]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.627386468952734

In [2332]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.6244986751621251

## Задание 3

In [2333]:
print(list(df.columns[np.argsort(-my_clf.feature_importances_)[-10:]]))
print(list(df.columns[np.argsort(-clf.feature_importances_)[-10:]]))

['attr', 'intel_o', 'condtn', 'prob', 'met', 'pf_o_sha', 'int_corr', 'age_o', 'met_o', 'intel']
['sinc_o', 'goal', 'fun_o', 'race_o', 'amb', 'condtn', 'met_o', 'met', 'samerace', 'gender']


## Задание 4

In [2336]:
param = {
    "min_samples_split": range(2, 60, 3),
    "max_depth": range(1, 20),
}
rf_clf = RandomForestClassifier(n_estimators=100)
rand_search = RandomizedSearchCV(rf_clf, param_distributions=param, n_iter=200)
%time rand_search.fit(X, y)
print(rand_search.best_params_)

CPU times: user 3min 8s, sys: 0 ns, total: 3min 8s
Wall time: 3min 8s
{'min_samples_split': 23, 'max_depth': 14}
